In [12]:
%ls ../data

Business_Licenses.csv


In [13]:
from pathlib import Path

import numpy as np
import pandas as pd

In [15]:
data_dir = Path("../data/")
df = pd.read_csv(data_dir/"Business_Licenses.csv")

df

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,LEGAL NAME,DOING BUSINESS AS NAME,ADDRESS,CITY,STATE,ZIP CODE,...,LICENSE TERM START DATE,LICENSE TERM EXPIRATION DATE,LICENSE APPROVED FOR ISSUANCE,DATE ISSUED,LICENSE STATUS,LICENSE STATUS CHANGE DATE,SSA,LATITUDE,LONGITUDE,LOCATION
0,22308-20060816,1723393,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,...,08/16/2006,08/15/2007,08/10/2006,08/11/2006,AAI,NaN,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
1,1620668-20160516,2455262,295026,1,BUCCI BIG & TALL INC.,BUCCI BIG & TALL INC.,558 W ROOSEVELT RD,CHICAGO,IL,60607,...,05/16/2016,05/15/2018,05/18/2016,08/30/2016,AAI,NaN,NaN,41.867339,-87.641590,"(41.86733856638269, -87.64159005699716)"
2,2368602-20160616,2460909,291461,3,"PROJECT: VISION , INC.","PROJECT : VISION , INC",2301 S ARCHER AVE 1 1,CHICAGO,IL,60616,...,06/16/2016,06/15/2017,06/21/2016,06/22/2016,AAC,08/30/2016,NaN,41.850843,-87.638734,"(41.85084294374687, -87.63873424399071)"
3,2060891-20141016,2353257,357247,1,FOLASHADE'S CLEANING SERVICE INC.,FOLASHADE'S CLEANING SERVICE INC.,1965 BERNICE RD 1 1SW,LANSING,IL,60438,...,10/16/2014,10/15/2016,04/01/2016,04/01/2016,AAI,NaN,38.0,41.951316,-87.678586,"(41.95131555606832, -87.67858578019546)"
4,1144216-20070516,1804790,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,...,05/16/2007,05/15/2008,05/10/2007,05/11/2007,AAI,NaN,5.0,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)"
5,2488345-20160830,2488345,409564,1,"BURKS HEATING AND COOLING SOLUTIONS, LLC","BURKS HEATING AND COOLING SOLUTIONS, LLC",5722 S WOOD ST 1ST,CHICAGO,IL,60636,...,08/30/2016,09/15/2018,08/30/2016,08/30/2016,AAI,NaN,NaN,41.789617,-87.669472,"(41.78961679000226, -87.66947199630827)"
6,22308-20030816,1479006,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,...,08/16/2003,08/15/2004,03/25/2004,04/16/2004,AAI,NaN,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
7,2354707-20160916,2482568,393149,1,ANGELINE R. MC CARTHY,ANGELINE R. MC CARTHY,17707 S HARLEM ST 1,TINLEY PARK,IL,60477,...,09/16/2016,09/15/2018,NaN,08/30/2016,AAI,NaN,NaN,NaN,NaN,NaN
8,22308-20010816,1151715,29481,1,BELL OIL TERMINAL INC,Bell Oil Terminal LLC,3741 S PULASKI RD 1,CHICAGO,IL,60623,...,08/16/2001,08/15/2002,08/10/2001,03/05/2004,AAI,NaN,NaN,41.825320,-87.723960,"(41.82531992987547, -87.72395999659746)"
9,1144216-20030516,1338337,147,63,WALGREEN CO.,Walgreens # 05192,9148 S COMMERCIAL AVE 1ST,CHICAGO,IL,60617,...,05/16/2003,05/15/2004,04/28/2003,04/28/2003,AAI,NaN,5.0,41.728622,-87.551366,"(41.72862173556932, -87.55136646594693)"


In [16]:
df.describe()

,LICENSE ID,ACCOUNT NUMBER,SITE NUMBER,WARD,PRECINCT,POLICE DISTRICT,LICENSE CODE,LICENSE NUMBER,SSA,LATITUDE,LONGITUDE
count,9.687450e+05,968745.000000,968745.000000,897358.000000,867531.000000,872897.000000,968745.000000,9.687440e+05,251612.000000,894393.000000,894393.000000
mean,1.859037e+06,177010.019276,4.475002,28.148523,28.539931,13.009229,1404.131445,1.174058e+06,29.793798,41.879976,-87.677243
std,4.204808e+05,140738.829495,17.808083,14.676520,47.791861,7.134794,1066.493991,8.855768e+05,17.201784,0.078202,0.056329
min,3.640000e+02,1.000000,1.000000,1.000000,1.000000,0.000000,1002.000000,2.000000e+00,1.000000,41.644497,-87.933977
25%,1.510431e+06,34155.000000,1.000000,16.000000,12.000000,8.000000,1010.000000,5.168200e+04,17.000000,41.836839,-87.711701
50%,1.839813e+06,210182.000000,1.000000,30.000000,26.000000,14.000000,1010.000000,1.357793e+06,29.000000,41.890902,-87.668276
75%,2.194026e+06,301699.000000,2.000000,42.000000,40.000000,18.000000,1470.000000,1.921710e+06,42.000000,41.937604,-87.634955
max,2.664142e+06,458500.000000,526.000000,50.000000,4047.000000,181.000000,8604.000000,2.664142e+06,69.000000,42.022660,-87.525094


In [37]:
df.head(5000).groupby(["ACCOUNT NUMBER", "LEGAL NAME"])["DATE ISSUED"].apply(list)

ACCOUNT NUMBER  LEGAL NAME                                        
13              BLEACHERS, INC.                                                                            [08/04/2004]
54              KONE INC.                                                                                  [02/27/2015]
82              LETTUCE ENTERTAIN YOU ENTERPRISES, INC.                                                    [05/03/2004]
116             FRANCES M MITCHELL                                                                         [11/15/2002]
147             WALGREEN CO.                                          [05/11/2007, 04/28/2003, 04/23/2004, 04/26/200...
175             LAPARA NIGHT CLUB, INC.                                                                    [06/15/2010]
259             JEWEL FOOD STORES, INC.                               [05/02/2005, 05/05/2004, 05/07/2007, 05/05/200...
465             MILLARD MAINTENANCE SERVICE CO                                               